# 1. 할리스 커피 매장 찾기
- http://www.hollys.co.kr

## HTML 코드 확인
- 매장 정보 확인
    - < t b o d y > 태그 내부
        - < t d > 태그 첫 번째 : 매장이 있는 지역
        - < t d > 태그 두 번째 : 매장 이름
        - < t d > 태그 네 번째 : 매장 주소
        - < t d > 태그 다섯 번째 : 매장 전화번호
- 나머지 매장 정보 확인
    - https://www.hollys.co.kr/store/korea/korStore2.do?pageNo=1&sido=&gugun=&store=

### 1. 매장 찾기에서 1~53페이지까지 모든 매장의 정보를 스크레이핑
- 지역, 매장명, 매장 주소, 전화번호
- 수집된 정보는 Pandas DataFrame으로 변경한 다음 csv 파일로 저장
- csv 파일 : hollys.branches.csv
- 소스 파일 : hw02.coffeeshop_to_csv.ipynb

### 2. 저장된 csv 파일을 읽어서 사용자가 입력한 지역에 있는 매장 정보 출력
- 소스 파일 : hw02_find_coffeeshop.ipynb

In [36]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

tds=[]     # <td> 태그 담는 리스트
result=[]  # <td> 태그의 text 담는 리스트

region=[]     # 지역을 담는 리스트
shop_name=[]  # 매장 이름을 담는 리스트
shop_addr=[]  # 매장 주소를 담는 리스트
call=[]       # 전화번호를 담는 리스트

for i in range(1, 54):  # 1~53페이지
    html=urlopen('https://www.hollys.co.kr/store/korea/korStore2.do?pageNo='+str(i)+'&sido=&gugun=&store=')
    soup=BeautifulSoup(html, 'html.parser')

    tbodys = soup.find_all('tbody')  # <tbody> 태그 찾기
    
    for tb in tbodys:
        tbody=tb.find_all('td')  # <tbody> 태그 안에서 <td> 태그 찾기
        for t in tbody:
            tds.append(t.text)   # <td> 태그에서 텍스트만 뽑아서 tds 리스트에 추가하기

for i in range(1, len(tds) + 1):
    if (i % 6 in [0, 1, 2, 4]):   # 지역, 매장명, 매장 주소, 전화번호만 뽑아서 result에 담기
        result.append(tds[i-1])


for i in range(len(result)):   # result에 있는 지역, 매장명, 매장 주소, 전화번호를 각각의 리스트에 나눠 담기
        if i%4==0:
            region.append(result[i])
        elif i%4==1:
            shop_name.append(result[i])
        elif i%4==2:
            shop_addr.append(result[i])
        else:
            call.append(result[i])


# 각 리스트를 데이터프레임으로 만들기
region_df=pd.DataFrame(region, columns=['지역'])
shop_name_df=pd.DataFrame(shop_name, columns=['매장명'])
shop_addr_df=pd.DataFrame(shop_addr, columns=['주소'])
call_df=pd.DataFrame(call, columns=['전화번호'])

# 각 데이터프레임을 하나의 데이터프레임으로 합치기
shop_info=pd.concat([region_df, shop_name_df, shop_addr_df, call_df], axis=1)

In [37]:
shop_info

지역          매장명                                                주소  \
0     서울 종로구         종각역점                      서울특별시 종로구 종로 60-1 종로2가 102-3   
1     부산 사하구  부산아트몰링영풍문고점               부산광역시 사하구 낙동남로 1413 (하단동, 아트몰링) 13층   
2     경기 광명시      광명GIDC점           경기도 광명시 일직로 43 일직동 514 지하2층 G249, G250호   
3     경기 이천시          이천점                    경기도 이천시 어재연로 35 (창전동) 1~3층 할리스   
4     경기 하남시      미사가로수길점        경기도 하남시 미사강변한강로 295 (망월동, 하남미사롯데캐슬헤븐시티Ⅱ) .   
..       ...          ...                                               ...   
525   서울 관악구          신림점                               서울특별시 관악구 신림로 353-1   
526    서울 중구         태평로점        서울특별시 중구 세종대로 64, 해남빌딩 1층 (태평로2가 70-5) 할리스   
527   서울 용산구         이태원점                               서울특별시 용산구  이태원로 179   
528  부산 부산진구       부산서면본점  부산광역시 부산진구 동천로 73, DS타워 1~2층 (부전동 부전동 169-1) 할리스   
529  서울 서대문구          신촌점                서울특별시 서대문구 연세로 34 (창천동 31-12)  할리스   

             전화번호  
0     02-736-3399  
1    051-201-0621  
2    02-6914-6550  
3    031-635-6655  
4    031-699-8674  
..            ...  
525   02-877-0019  
526   02-755-7795  
527   02-749-8752  
528  051-819-9117  
529   02-393-2004  

[530 rows x 4 columns]

In [38]:
num=shop_info.shape[0]  # shop_info의 행의 수
for i in range(1, num+1):
    print('[ {}]: 매장이름: {}, 지역: {}, 주소: {}, 전화번호: {}'.format(i, shop_name[i-1], region[i-1], shop_addr[i-1], call[i-1]))

print('='*120)
print('전체 매장 수 :', num)

[ 1]: 매장이름: 종각역점, 지역: 서울 종로구, 주소: 서울특별시 종로구 종로 60-1 종로2가 102-3, 전화번호: 02-736-3399
[ 2]: 매장이름: 부산아트몰링영풍문고점, 지역: 부산 사하구, 주소: 부산광역시 사하구 낙동남로 1413 (하단동, 아트몰링) 13층, 전화번호: 051-201-0621
[ 3]: 매장이름: 광명GIDC점, 지역: 경기 광명시, 주소: 경기도 광명시 일직로 43 일직동 514 지하2층 G249, G250호, 전화번호: 02-6914-6550
[ 4]: 매장이름: 이천점, 지역: 경기 이천시, 주소: 경기도 이천시 어재연로 35 (창전동) 1~3층 할리스, 전화번호: 031-635-6655
[ 5]: 매장이름: 미사가로수길점, 지역: 경기 하남시, 주소: 경기도 하남시 미사강변한강로 295 (망월동, 하남미사롯데캐슬헤븐시티Ⅱ) ., 전화번호: 031-699-8674
[ 6]: 매장이름: 시흥능곡점, 지역: 경기 시흥시, 주소: 경기도 시흥시 능곡번영길 25 (능곡동) 능곡동 211-3 (107,108,205,206호), 전화번호: 070-8806-1314
[ 7]: 매장이름: 송탄서정점, 지역: 경기 평택시, 주소: 경기도 평택시 경기대로 1371,102호 서정동 814-1, 전화번호: 031-611-0234
[ 8]: 매장이름: 장승배기역점, 지역: 서울 동작구, 주소: 서울특별시 동작구 장승배기로11가길 11, 1층 (상도동, 상도파크자이), 전화번호: 02-812-9330
[ 9]: 매장이름: 미사효성해링턴점, 지역: 경기 하남시, 주소: 경기도 하남시 아리수로 570,1~2층 망월동 미사효성해링턴타워더퍼스, 전화번호: 070-7733-9710
[ 10]: 매장이름: 이천마장점, 지역: 경기 이천시, 주소: 경기도 이천시 마장면 오천로 65 오천리 56-55, 전화번호: 070-4647-0081
[ 11]: 매장이름: 김해인제대점, 지역: 경남 김해시, 주소: 경상남도 김해시 인제로 168 (어방동) 정석빌딩

In [39]:
# shop_info 데이터프레임을 csv 파일로 변환
shop_info.to_csv('hollys_branches.csv', index=False, encoding='utf-8')